In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import dill as pickle
import bisect
import torch
import re
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

from matplotlib import animation, rc
rc('animation', html='html5')

import sys
sys.path.append('..')
from util import add_angles, angle_between, angled_vector, clip_angle, unit_vector, x_axis, get_rotation_matrix
import binning
import data
from plots import *
from social_models import load_social_model
import segmentation


cuda:0
cuda:0


In [2]:
X_train, y_train, X_test, y_test = [d.to(device) for d in data.get_data('../../data/processed/')]

In [3]:
X_train.shape, X_test.shape, y_train.shape

(torch.Size([8, 19427, 192]),
 torch.Size([8, 4436, 192]),
 torch.Size([19427, 2]))

In [4]:
def multiple_replace(string, replacements):
    """
    Given a string and a replacement map, it returns the replaced string.
    :param str string: string to execute replacements on
    :param dict replacements: replacement dictionary {value to find: value to replace}
    :rtype: str
    
    Modified from: https://gist.github.com/bgusach/a967e0587d6e01e889fd1d776c5f3729
    """
    substrs = sorted(replacements, key=len, reverse=True)

    # Create a big OR regex that matches any of the substrings to replace
    regexp = re.compile('|'.join(map(re.escape, substrs)))

    # For each match, look up the new string in the replacements
    return regexp.sub(lambda match: replacements[match.group(0)], string)


def interpolate_time(df, required_timesteps):
    def interpolate_col(data, time, new_time):
        # Piece-wise linear interpolation.
        interpolated = np.interp(x=new_time, fp=data, xp=time)

        return pd.Series(interpolated)

    # Save columns. Otherwise we loose information about dropped frames!
    time = np.copy(df['time'].values)

    new_time = time.max() - required_timesteps
    print(time, '\n', new_time)
    
    # Make sure time is increasing
    assert(np.all(np.diff(time) > 0.0))
    
    # Resample entries.
    df = df.apply((lambda col: interpolate_col(col, time, new_time)), axis=0)
    
    # Restore time column.
    df.loc[:, 'time'] = new_time
       
    # Convert time column to dt
    df.loc[:, 'dt'] = pd.Series(np.uint((required_timesteps)*100), index=df.index)
    
    return df

def boundary_df(df, bounding_box):
    def apply_boundary_row(row):
        # wall distances are [yMin, xMax, yMax, xMin]
        # note that each position can be interpreted as three pos.
        # one reached directly, the other two reached via walls
        # (due to periodic boundary conditions)
        # We choose the one with the smallest distance.
        x_min, x_max, y_min, y_max = bounding_box
        x_pos = (row['x_f1'],
                 x_max + row['wall_distance1_f1'],
                 x_min - row['wall_distance3_f1'])
        y_pos = (row['y_f1'],
                 y_max + row['wall_distance0_f1'],
                 y_min - row['wall_distance2_f1'])
        min_dist = np.double('inf')
        min_pos = None
        for x in x_pos:
            for y in y_pos:
                pos_0, pos_1 = np.array([row['x_f0'], row['y_f0']]), np.array([x, y])
                dist = np.linalg.norm(pos_0 - pos_1)
                if dist < min_dist:
                    min_dist = dist
                    min_pos = pos_1

        row['x_f1'], row['y_f1'] = min_pos
        return row
    
    return df.apply(apply_boundary_row, axis=1)

class History(object):
    def __init__(self, store_minimum=0.5):
        self.store_minimum = store_minimum
        
        self.data = [[],[]]
        self.time = [[], []]
        self.last_kick = [0, 0]
        
    def add(self, fish_id, time, data, is_kick=False):
        #assert(self.time[fish_id][-1] < time)           
        self.data[fish_id].append(data)
        self.time[fish_id].append(np.around(time, decimals=6))
        
        if is_kick:
            self.record_kick(fish_id, time)
            
        assert(len(self.data[fish_id]) == len(self.time[fish_id]))
                
    def record_kick(self, fish_id, time):
        """Sets last kick for fish id and resets buffer for other fish"""
        #time_kick_before = self.last_kick[fish_id]
        self.last_kick[fish_id] = self.time
        
        other_id = 1 - fish_id
        # Clean buffer for other fish if needed.
        if self.time[1-fish_id]:
            end = self.time[other_id][-1]
            start = self.time[other_id][0]
            elapsed = end - start
            
            if elapsed > 2 * self.store_minimum:
                # Find idx where we can cutoff
                idx_time = bisect.bisect(self.time[other_id], end - self.store_minimum)
                print(len(self.data[other_id]))
                self.data[other_id] = self.data[other_id][idx_time:]
                print(len(self.data[other_id]))
                self.time[other_id] = self.time[other_id][idx_time:]

    def get(self, required_timesteps, edges, fish_id, world_size, return_df=False):
        time = [{'time': t} for t in self.time[0]]
        history = [{**t, **a, **b} for a,b,t in zip(self.data[:][0], self.data[:][1], time)]
        df = pd.DataFrame(history, index=None)
        
        assert(df.size > 0 )
        # For all later usages, f0 is assumed to be the focal fish.
        # -> If fish_id is different, exchange both.
        #print(df)
        if fish_id != 'f0':
            replacements = {'f0': 'f1', 'f1':'f0'}
            df = df.rename(columns=lambda col: multiple_replace(col, replacements))

        # Get required timesteps by linear interpolation
        # If stepsize of simulation is equal to history stepsize, this doesn't do anything!
        df = interpolate_time(df=df, 
                              required_timesteps=required_timesteps)
        # Not used for predictions
        df.loc[:, 'heading_change'] = pd.Series(float('nan'), index=df.index)
        df.loc[:, 'length'] = pd.Series(float('nan'), index=df.index)
         
        # Apply periodic boundary conditions
        bounding_box = np.array([0, world_size[0], 0, world_size[1]])
        df = boundary_df(df, bounding_box)
        
        if return_df is True:
            return df

        # Convert to local coordinate system:
        df = binning.transform_coords_df(df, cutoff_wall_range=None)

        # Discretize
        df = binning.get_bins_df(df=df, **edges)

        # Normalize
        # Means, stds are from training set
        means, stds = (0.6348294576188627, 0.0012032019097761566), (0.5596922160565242, 0.5326711711690991)

        # Values of y are NaN (they are not needed!)
        X, y = binning.get_Xy(df=df,
                  num_bins=edges['num_bins'],
                  means=means,
                  stds=stds)
        rf_df = binning.Xy_to_df(X, y)

        # Reshape according to our conventions
        X, y = data.process_df(rf_df)

        return X.to(device)
    
with open('../../models/adaptive_bins.model', 'rb') as f:
    edges = pickle.load(f)
    
with open('../../models/kick_duration.model', 'rb') as f:
    kick_duration = pickle.load(f)   

In [5]:
class Fish(object):
    def __init__(self, fish_id, kick_model, world_size, boundary_condition):
        self.fish_id = fish_id
        self.kick_model = kick_model
        self.world_size = world_size
        self.boundary_condition = boundary_condition
        
        self.kick_position_start = self.world_size//2 + np.random.random(size=2) * self.world_size//2
        self.kick_position_end = self.kick_position_start + np.array([0.5, 0])
        
        self.kick_time_start = 0
        self.kick_time_end = 0
        self.time = 0
        
    def kick(self, history):
        print(f'{self.fish_id}\t{self.time}\tKicked')
               
        kick_trajectory, kick_duration = self.kick_model(fish_id=self.fish_id,
                       history=history,
                       state=self.get_state())
        
        self.kick_time_start = self.time
        self.kick_time_end = self.time + kick_duration
        # Apply bc to kick start and NOT to kick end.
        # This way we can simply interpolate between start and end!
        self.kick_position_start = self.boundary_condition(self.kick_position_end)
        self.kick_position_end = self.kick_position_start + kick_trajectory

    def step(self, 
             dt,
             history,
             boundary_condition=lambda x:x):
        eps = 1e-6 # to compare floats
        is_kick = False
        
        end_time = self.time + dt
        while dt > eps:
            # Move until length of current kick
            cur_time = min(self.kick_time_end, self.time + dt)
            dt -= cur_time - self.time
            self.time = cur_time

            if dt > eps:
                # We need to kick off here.
                self.kick(history=history)
                
                # Make sure that we didn't kick off before:
                assert(not is_kick)
                is_kick = True                
                
        # Avoid float inprecision!
        # This makes sure that both fish have same time at all history points.
        self.time = end_time
        
        history.add(fish_id=self.fish_id,
                   time=self.time,
                   data=self.get_state(),
                   is_kick=is_kick)        
        
    def get_pos(self):
        # Linear interpolation between end and start
        pos_dt = self.kick_position_end - self.kick_position_start
        kick_time = self.kick_time_end - self.kick_time_start
        elapsed_time = self.time - self.kick_time_start
        
        if kick_time == 0:
            weight = 0
        else:
            weight = elapsed_time/kick_time
        return self.boundary_condition((1-weight) * self.kick_position_start
                            + weight *self.kick_position_end)
    
    def get_state(self):
        x, y = self.get_pos()
        
        kick_trajectory = self.kick_position_end - self.kick_position_start
        heading_change = angle_between(x_axis, kick_trajectory)
        heading = angle_between(x_axis, kick_trajectory)

        kick_length = np.linalg.norm(kick_trajectory)
        
        bounding_box = np.array([0, self.world_size[0], 0, self.world_size[1]])
        distances, _ = segmentation.get_wall_influence(orientation=heading,
                                                      point=np.array([x,y]),
                                                      bounding_box=bounding_box)
        return {f'heading_change_f{self.fish_id}': heading_change,
                f'length_f{self.fish_id}': kick_length,
                f'angle_f{self.fish_id}': heading,
                f'x_f{self.fish_id}': x,
                f'y_f{self.fish_id}': y,
                # if using wall model, also add wall angle to state!
                f'wall_distance0_f{self.fish_id}': distances[0],
                f'wall_distance1_f{self.fish_id}': distances[1],
                f'wall_distance2_f{self.fish_id}': distances[2],
                f'wall_distance3_f{self.fish_id}': distances[3],
               }

class WorldState(object):
    def __init__(self, size, kick_model):
        self.size = size
        # Use periodic boundary conditions
        boundary_condition = lambda pos: np.mod(pos, self.size)
            
        self.time = 0
        self.time_dt = 0.01 # time step from experiment
        self.history_dt = 5 # frames between history snapshots
        self.history_size = 15 # keep n snapshots
        self.fish = [Fish(fish_id=0,
                        boundary_condition=boundary_condition,
                        kick_model=kick_model,
                        world_size=size),
                     Fish(fish_id=1,
                        boundary_condition=boundary_condition,
                        kick_model=kick_model,
                        world_size=size)]
        self.history = History()
        
        # Init history with initial pos.
        # otherwise df is zero for beginning!
        for i,f in enumerate(self.fish):
            self.history.add(fish_id=i,
                        time=self.time,
                        data=f.get_state())
    
    def step(self, dt=None):
        if dt is None:
            dt = self.time_dt
        
        # Pass the history of the other fish to each fish
        for fish in self.fish:
            has_kicked = fish.step(dt, history=self.history)
            
        # Round time, not used for simulation anyway
        self.time = np.around(self.time + dt, decimals=4)
        print(f"Time: {self.time}")
    
    def get_pos(self):
        return np.array([self.fish[0].get_pos(), 
                        self.fish[1].get_pos()])  
   

class KickModel(object):
    def __init__(self, social_model, duration_model, world_size, wall_model=None):
        self.social_model = social_model
        self.wall_model = wall_model
        self.duration_model = duration_model
        self.world_size = world_size
        assert(self.wall_model is None) # not supported
    
    # todo: pass edges
    def __call__(self, fish_id, state, history,):
        # Note that the receptive field does not exhibit the periodic boundary
        # conditions!
        receptive_field = history.get(
            required_timesteps=self.social_model.get_required_timesteps()/100,
            edges=edges,
            world_size=world_size,
            fish_id=fish_id)
        current_angle = state[f'angle_f{fish_id}']
        
        kick_trajectory = self.social_model.sample(receptive_field).reshape(-1)
        
        # Predict new trajectory in local coordinate system
        # (where fish is at (0,0) and has angle 0        
        kick_duration = self.duration_model.sample()[0][0,0]

        # Rotate vector - prediction was in local rf coordinate system
        rotation_matrix = get_rotation_matrix(current_angle)

        kick_trajectory = rotation_matrix @ kick_trajectory
        
        return kick_trajectory, kick_duration
    
    
    def get_required_timesteps(self):
        return social_model.get_required_timesteps()
  

social_model = load_social_model('../../models/rnn_mdn.pt', X_train, y_train)
world_size = np.array([30,30])
kick_model = KickModel(social_model=social_model,
                      duration_model=kick_duration,
                      world_size=world_size)
print(social_model.get_required_timesteps())

world = WorldState(size=world_size, 
                   kick_model=kick_model)

<class 'int'>
{'no_memory': array([0]), 'memory': array([ 0.        ,  5.71428571, 11.42857143, 17.14285714, 22.85714286,
       28.57142857, 34.28571429, 40.        ])}
[ 0.  5. 10. 15. 20. 25. 30. 35.]


In [6]:
def add_to_buffer(buffer, value):
    buffer_local = np.roll(buffer, shift=-1)
    buffer_local[-1] = value
    np.copyto(dst=buffer, src=buffer_local)
    return buffer

fig = plt.figure(figsize=(10,10))
ax = plt.axes(xlim=(0, world.size[0]), ylim=(0, world.size[1]))
plt.close(fig)

lines = [None] * 2
lines[0], = ax.plot([], [], c='red', linewidth=5, label='fish 1')
lines[1], = ax.plot([], [], c='gray', linewidth=5, label='fish 2')

ax.legend(loc='upper right')

# Set up animation buffers.
visible_steps = 20

# Shape of buffer: fish_id, coord, step
animation_buffer = np.ones((2,2,visible_steps))

# Init buffer with starting position of fish
animation_buffer[0,0,:] *= world.get_pos()[0][0]
animation_buffer[0,1,:] *= world.get_pos()[0][1]
animation_buffer[1,0,:] += world.get_pos()[1][0]
animation_buffer[1,1,:] *= world.get_pos()[1][1]

animation_frames = 2000//2
animation_interval = 40//2
animation_dt = animation_interval/animation_frames
print(animation_dt)

def init():
    for line in lines:
        line.set_data([], [])
    return lines[0], lines[1]

def animate(i):
    world.step(animation_dt)
    cur_positions = world.get_pos()
    #print(cur_positions)
    
    for i, f in enumerate(world.fish):
        break
        state = f.get_state()
        x, y = state[f'x_f{i}'], state[f'x_f{i}']
        angle = state[f'angle_f{i}']
        traj_x, traj_y = angled_vector(angle)*2
        ax.arrow(x, y, traj_x, traj_y, width=0.2, color='gray')
        ax.scatter(x,y, s=300, marker='x', linewidth=2)
    
    for fish_id, position in enumerate(cur_positions):
        # Update animation buffers
        add_to_buffer(animation_buffer[fish_id, 0], position[0])
        add_to_buffer(animation_buffer[fish_id, 1], position[1])
        
        data_0 = animation_buffer[fish_id, 0]
        data_1 = animation_buffer[fish_id, 1]
        
        # Don't draw data before 'jump' due to boundary cond!
        cond = (np.abs(data_0 - data_0[-1]) < 2) & (np.abs(data_1 - data_1[-1]) < 2)
        data_0 = data_0[cond]
        data_1 = data_1[cond]
        
        # Update graphic
        lines[fish_id].set_data(data_0,
                                data_1)
        
        #lines[fish_id].set_data(animation_buffer[fish_id, 0],
        #                        animation_buffer[fish_id, 1])

    return lines[0], lines[1], 

anim = animation.FuncAnimation(fig,
                               animate,
                               init_func=init,
                               frames=animation_frames,
                               interval=animation_interval,
                               blit=True,)

#anim

0.02


In [7]:
anim.save('../../figures/social_anim.mp4', dpi=92*2, fps=animation_frames/20)

0	0	Kicked
[0] 
 [ 0.   -0.05 -0.1  -0.15 -0.2  -0.25 -0.3  -0.35]
1	0	Kicked
[0] 
 [ 0.   -0.05 -0.1  -0.15 -0.2  -0.25 -0.3  -0.35]
Time: 0.02
Time: 0.04
Time: 0.06
Time: 0.08
Time: 0.1
Time: 0.12
Time: 0.14
Time: 0.16
Time: 0.18
Time: 0.2
Time: 0.22
Time: 0.24
Time: 0.26
Time: 0.28
Time: 0.3
Time: 0.32
0	0.3241433116673979	Kicked
[0.   0.02 0.04 0.06 0.08 0.1  0.12 0.14 0.16 0.18 0.2  0.22 0.24 0.26
 0.28 0.3  0.32] 
 [ 0.32  0.27  0.22  0.17  0.12  0.07  0.02 -0.03]
Time: 0.34
1	0.35156010923481346	Kicked
[0.   0.02 0.04 0.06 0.08 0.1  0.12 0.14 0.16 0.18 0.2  0.22 0.24 0.26
 0.28 0.3  0.32 0.34] 
 [ 0.34  0.29  0.24  0.19  0.14  0.09  0.04 -0.01]
Time: 0.36
Time: 0.38
Time: 0.4
Time: 0.42
Time: 0.44
Time: 0.46
Time: 0.48
Time: 0.5
Time: 0.52
Time: 0.54
Time: 0.56
Time: 0.58
Time: 0.6
Time: 0.62
1	0.6320483861866828	Kicked
[0.   0.02 0.04 0.06 0.08 0.1  0.12 0.14 0.16 0.18 0.2  0.22 0.24 0.26
 0.28 0.3  0.32 0.34 0.36 0.38 0.4  0.42 0.44 0.46 0.48 0.5  0.52 0.54
 0.56 0.58 0.6  0.6

Time: 3.9
Time: 3.92
Time: 3.94
1	3.957555031387633	Kicked
[3.26 3.28 3.3  3.32 3.34 3.36 3.38 3.4  3.42 3.44 3.46 3.48 3.5  3.52
 3.54 3.56 3.58 3.6  3.62 3.64 3.66 3.68 3.7  3.72 3.74 3.76 3.78 3.8
 3.82 3.84 3.86 3.88 3.9  3.92 3.94 3.96] 
 [3.96 3.91 3.86 3.81 3.76 3.71 3.66 3.61]
Time: 3.96
Time: 3.98
Time: 4.0
Time: 4.02
Time: 4.04
0	4.053944884965367	Kicked
[3.26 3.28 3.3  3.32 3.34 3.36 3.38 3.4  3.42 3.44 3.46 3.48 3.5  3.52
 3.54 3.56 3.58 3.6  3.62 3.64 3.66 3.68 3.7  3.72 3.74 3.76 3.78 3.8
 3.82 3.84 3.86 3.88 3.9  3.92 3.94 3.96 3.98 4.   4.02 4.04] 
 [4.04 3.99 3.94 3.89 3.84 3.79 3.74 3.69]
Time: 4.06
Time: 4.08
Time: 4.1
Time: 4.12
Time: 4.14
Time: 4.16
Time: 4.18
Time: 4.2
Time: 4.22
Time: 4.24
Time: 4.26
Time: 4.28
Time: 4.3
Time: 4.32
1	4.336745602593801	Kicked
[3.26 3.28 3.3  3.32 3.34 3.36 3.38 3.4  3.42 3.44 3.46 3.48 3.5  3.52
 3.54 3.56 3.58 3.6  3.62 3.64 3.66 3.68 3.7  3.72 3.74 3.76 3.78 3.8
 3.82 3.84 3.86 3.88 3.9  3.92 3.94 3.96 3.98 4.   4.02 4.04 4.06 4

Time: 8.26
Time: 8.28
Time: 8.3
Time: 8.32
Time: 8.34
Time: 8.36
Time: 8.38
Time: 8.4
Time: 8.42
Time: 8.44
Time: 8.46
Time: 8.48
0	8.487029218649822	Kicked
[7.3  7.32 7.34 7.36 7.38 7.4  7.42 7.44 7.46 7.48 7.5  7.52 7.54 7.56
 7.58 7.6  7.62 7.64 7.66 7.68 7.7  7.72 7.74 7.76 7.78 7.8  7.82 7.84
 7.86 7.88 7.9  7.92 7.94 7.96 7.98 8.   8.02 8.04 8.06 8.08 8.1  8.12
 8.14 8.16 8.18 8.2  8.22 8.24 8.26 8.28 8.3  8.32 8.34 8.36 8.38 8.4
 8.42 8.44 8.46 8.48] 
 [8.48 8.43 8.38 8.33 8.28 8.23 8.18 8.13]
64
25
Time: 8.5
Time: 8.52
Time: 8.54
Time: 8.56
Time: 8.58
Time: 8.6
Time: 8.62
Time: 8.64
Time: 8.66
1	8.667345520425961	Kicked
[7.3  7.32 7.34 7.36 7.38 7.4  7.42 7.44 7.46 7.48 7.5  7.52 7.54 7.56
 7.58 7.6  7.62 7.64 7.66 7.68 7.7  7.72 7.74 7.76 7.78 7.8  7.82 7.84
 7.86 7.88 7.9  7.92 7.94 7.96] 
 [7.96 7.91 7.86 7.81 7.76 7.71 7.66 7.61]
70
25
Time: 8.68
Time: 8.7
Time: 8.72
Time: 8.74
Time: 8.76
Time: 8.78
Time: 8.8
0	8.819928776273485	Kicked
[8.2  8.22 8.24 8.26 8.28 8.3  8.32 8.

62
25
Time: 12.0
Time: 12.02
Time: 12.04
Time: 12.06
Time: 12.08
Time: 12.1
Time: 12.12
Time: 12.14
Time: 12.16
Time: 12.18
Time: 12.2
Time: 12.22
Time: 12.24
1	12.253704672140264	Kicked
[11.5  11.52 11.54 11.56 11.58 11.6  11.62 11.64 11.66 11.68 11.7  11.72
 11.74 11.76 11.78 11.8  11.82 11.84 11.86 11.88 11.9  11.92 11.94 11.96
 11.98 12.   12.02 12.04 12.06 12.08 12.1  12.12 12.14 12.16 12.18 12.2
 12.22 12.24] 
 [12.24 12.19 12.14 12.09 12.04 11.99 11.94 11.89]
Time: 12.26
Time: 12.28
Time: 12.3
0	12.307791089713803	Kicked
[11.5  11.52 11.54 11.56 11.58 11.6  11.62 11.64 11.66 11.68 11.7  11.72
 11.74 11.76 11.78 11.8  11.82 11.84 11.86 11.88 11.9  11.92 11.94 11.96
 11.98 12.   12.02 12.04 12.06 12.08 12.1  12.12 12.14 12.16 12.18 12.2
 12.22 12.24 12.26 12.28 12.3 ] 
 [12.3  12.25 12.2  12.15 12.1  12.05 12.   11.95]
Time: 12.32
Time: 12.34
Time: 12.36
Time: 12.38
Time: 12.4
Time: 12.42
Time: 12.44
Time: 12.46
Time: 12.48
Time: 12.5
1	12.515285000597496	Kicked
[11.5  11.52 11.54

Time: 15.4
Time: 15.42
Time: 15.44
Time: 15.46
1	15.477201257443438	Kicked
[14.46 14.48 14.5  14.52 14.54 14.56 14.58 14.6  14.62 14.64 14.66 14.68
 14.7  14.72 14.74 14.76 14.78 14.8  14.82 14.84 14.86 14.88 14.9  14.92
 14.94 14.96 14.98 15.   15.02 15.04 15.06] 
 [15.06 15.01 14.96 14.91 14.86 14.81 14.76 14.71]
52
25
Time: 15.48
Time: 15.5
Time: 15.52
Time: 15.54
Time: 15.56
Time: 15.58
Time: 15.6
Time: 15.62
Time: 15.64
Time: 15.66
Time: 15.68
Time: 15.7
Time: 15.72
1	15.730049736653038	Kicked
[15.   15.02 15.04 15.06 15.08 15.1  15.12 15.14 15.16 15.18 15.2  15.22
 15.24 15.26 15.28 15.3  15.32 15.34 15.36 15.38 15.4  15.42 15.44 15.46
 15.48 15.5  15.52 15.54 15.56 15.58 15.6  15.62 15.64 15.66 15.68 15.7
 15.72 15.74] 
 [15.74 15.69 15.64 15.59 15.54 15.49 15.44 15.39]
Time: 15.74
Time: 15.76
Time: 15.78
Time: 15.8
Time: 15.82
Time: 15.84
Time: 15.86
Time: 15.88
Time: 15.9
Time: 15.92
Time: 15.94
0	15.958525717637688	Kicked
[15.   15.02 15.04 15.06 15.08 15.1  15.12 15.14 15.16

Time: 18.72
Time: 18.74
Time: 18.76
Time: 18.78
Time: 18.8
Time: 18.82
1	18.83588283100577	Kicked
[17.88 17.9  17.92 17.94 17.96 17.98 18.   18.02 18.04 18.06 18.08 18.1
 18.12 18.14 18.16 18.18 18.2  18.22 18.24 18.26 18.28 18.3  18.32 18.34
 18.36 18.38 18.4  18.42 18.44 18.46 18.48 18.5  18.52 18.54 18.56 18.58
 18.6  18.62 18.64 18.66 18.68 18.7  18.72 18.74] 
 [18.74 18.69 18.64 18.59 18.54 18.49 18.44 18.39]
Time: 18.84
Time: 18.86
Time: 18.88
Time: 18.9
Time: 18.92
Time: 18.94
Time: 18.96
Time: 18.98
Time: 19.0
Time: 19.02
Time: 19.04
Time: 19.06
0	19.079101483229984	Kicked
[17.88 17.9  17.92 17.94 17.96 17.98 18.   18.02 18.04 18.06 18.08 18.1
 18.12 18.14 18.16 18.18 18.2  18.22 18.24 18.26 18.28 18.3  18.32 18.34
 18.36 18.38 18.4  18.42 18.44 18.46 18.48 18.5  18.52 18.54 18.56 18.58
 18.6  18.62 18.64 18.66 18.68 18.7  18.72 18.74 18.76 18.78 18.8  18.82
 18.84 18.86 18.88 18.9  18.92 18.94 18.96 18.98] 
 [18.98 18.93 18.88 18.83 18.78 18.73 18.68 18.63]
56
25
Time: 19.08
T